In [ ]:
pip install duckduckgo_search langchain_groq sentence_transformers faiss-cpu pydantic python-dotenv pypdf bs4 langchain langchain_core langchain-community langchain_huggingface langchain_google_genai langchain-pinecone langgraph langchain-openai langchain_groq langchain_google_genai google-generativeai  pymupdf wikipedia arxiv defusedxml  youtube_search langchain-tavily langgraph chromadb


In [12]:
#### import
import os
import pprint
from dotenv import load_dotenv
from langgraph.graph import Graph
from IPython.display import Image, display
load_dotenv()


True

In [13]:
### Configure the Model
from langchain_groq import ChatGroq
model = ChatGroq(model="deepseek-r1-distill-llama-70b")
query = "Hi"
response = model.invoke(query)
pprint.pprint(response.content)



'<think>\n\n</think>\n\nHello! How can I assist you today? 😊'


In [14]:
### Configure the Embedding
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
embed_query = embeddings.embed_query(query)
len(embed_query)


384

In [16]:
# Lets take a Data embedd it and store in Vector Database
##### Imports
import requests
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import operator
from langchain_core.tools import tool
from langchain.tools import tool
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,END,MessagesState,START
from langgraph.prebuilt import ToolNode

In [45]:
# Travel Agent Tool
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import re

class TravelAgent:
    """Handles general travel planning queries."""
    @tool
    def travel_agent(query:str) -> str:
        """Handles general travel planning queries."""
        return f"Travel Agent : I can help plan your trip. You asked : {query} "
    
class WeatherAgent:
    
    def __init__(self, api_key):
        
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5/weather"
    
    def get_weather_city(self, city: str) -> str:
        """Provides weather information for a city."""
        params = {
        "q" : city,
        "appid" : self.api_key,
        "units" : "metric"
        }
        response = requests.get(self.base_url, params=params)    
        if response.status_code == 200:
                
            data = response.json()
            weather_desc = data["weather"][0]["description"]
            temp = data["main"]["temp"]
            humidity = data["main"]["humidity"]
            wind_speed = data["wind"]["speed"]
            return {
                "city": city,
                "description": weather_desc,
                "temperature_celsius": temp,
                "humidity": humidity,
                "wind_speed_m_s": wind_speed
            }
        else:
            return {"error": f"Coold not retrieve weather data for {city}"}
        #weather = Weather(api_key=weather_api_key)
        return f"Weather Agent: The Weather is {city} is sunny and 30 degrees Celsius."
                

class AttrationAgent:
    search = DuckDuckGoSearchRun()
    @tool
    def get_attraction_details(city: str) -> str:
        """Provides top attractions for a city."""
        city = city
        query = f"top 5 tourist attractions in {city}"
        results = AttrationAgent.search(query)
        # Use regex to extract lines that look like "1. Place Name" or similar
        place_names = re.findall(r'\d+\.\s*([A-Za-z0-9\s\'\-&]+)', results)
        # Remove duplicates and strip whitespace
        unique_places = []
        for name in place_names:
            clean_name = name.strip()
            if clean_name and clean_name not in unique_places:
                unique_places.append(clean_name)
        # Take top 5
        top_places = unique_places[:5]
        if top_places:
            return f"Attraction Agent: The top 5 attractions in {city} are: {', '.join(top_places)}."
        else:
            return f"Attraction Agent: Sorry, couldn't extract attractions for {city}."
        
    
class CurrencyAgent:
    @tool
    def get_currency_exchange_rate(currency: str) -> str:
        """Provides currency exchange rates."""
        return f"Currency Agent: The current exchange rate for {currency} is 1 {currency} = 1.2 USD."
    
search = DuckDuckGoSearchRun()
# search.invoke("What is the latest update on iPhone 17 relase ?")

# Register all agents/tools
mytools = [
    TravelAgent.travel_agent, 
    WeatherAgent.get_weather_city, 
    AttrationAgent.get_attraction_details, 
    CurrencyAgent.get_currency_exchange_rate, 
    search
]

llm_Trip_Agent = model.bind_tools(mytools)

SYSTEM_PROMPT = (
    "You are a helpful Travel Planner Agent. "
    "You can use the following agents: travel_agent, weather_agent, attraction_agent, currency_converter, search. "
    "Route the user's query to the correct agent(s) and combine their responses as needed."
)

# Supervisor orchestrates the workflow
def supervisor(state:MessagesState):
    USER_QUERY = state["messages"]
    INPUT_QUERY = [SYSTEM_PROMPT]+USER_QUERY

    response = llm_Trip_Agent.invoke(INPUT_QUERY)

    return {
        "messages" : [response]
    }


In [46]:

# Build the workflow graph
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display

TravelAgent_workflow_builder = StateGraph(MessagesState)
TravelAgent_workflow_builder.add_node("Travel Planner Agent", supervisor)
TravelAgent_workflow_builder.add_node("tools", ToolNode(mytools))
TravelAgent_workflow_builder.add_edge(START, "Travel Planner Agent")
TravelAgent_workflow_builder.add_conditional_edges("Travel Planner Agent", tools_condition,)
TravelAgent_workflow_builder.add_edge("tools","Travel Planner Agent")
## React and Action
react_graph = TravelAgent_workflow_builder.compile()

In [48]:
message = [HumanMessage(content="What are the top attractions and weather in Hyderabad?")]
response = react_graph.invoke({"messages": message})

for m in response["messages"]:
    m.pretty_print()


================================ Human Message =================================

What are the top attractions and weather in Hyderabad?
================================== Ai Message ==================================
Tool Calls:
  get_attraction_details (n21evgamq)
 Call ID: n21evgamq
  Args:
    city: Hyderabad
================================= Tool Message =================================
Name: get_attraction_details

Error: DuckDuckGoSearchException(RatelimitException('https://html.duckduckgo.com/html 202 Ratelimit'))
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  get_attraction_details (544awbtf2)
 Call ID: 544awbtf2
  Args:
    city: Hyderabad
================================= Tool Message =================================
Name: get_attraction_details

Error: DuckDuckGoSearchException(RatelimitException('https://html.duckduckgo.com/html 202 Ratelimit'))
 Please fix your mistakes.
=========================

In [9]:
# Example usage
message = [HumanMessage(content="what is 2 times of a nrendraamodi's age ?")]
response = react_graph.invoke({"messages": message})
response

{'messages': [HumanMessage(content="what is 2 times of a nrendraamodi's age ?", additional_kwargs={}, response_metadata={}, id='78ff56e0-3a03-4da7-8e1c-60f6fd6eb65a'),
  AIMessage(content="I'm sorry, but I can't provide that information. However, I can help with other travel-related queries you might have!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 329, 'prompt_tokens': 359, 'total_tokens': 688, 'completion_time': 1.471695021, 'prompt_time': 0.029899081, 'queue_time': 0.052790579, 'total_time': 1.5015941019999999}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--86c85380-0b19-4eb2-8a14-da4d5682cdc7-0', usage_metadata={'input_tokens': 359, 'output_tokens': 329, 'total_tokens': 688})]}

In [10]:
# Example usage
message = [HumanMessage(content="How much is the net worth of elon musk and divided by 2 ?")]
response = react_graph.invoke({"messages": message})
response

{'messages': [HumanMessage(content='How much is the net worth of elon musk and divided by 2 ?', additional_kwargs={}, response_metadata={}, id='6e495594-a60d-4d7a-87f4-78ca49461b22'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'km0ff2kpc', 'function': {'arguments': '{"query":"Elon Musk net worth 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 122, 'prompt_tokens': 361, 'total_tokens': 483, 'completion_time': 0.494315419, 'prompt_time': 0.030296724, 'queue_time': 0.051432935, 'total_time': 0.524612143}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--46469eb2-d7c5-4884-90b1-10f319c903e3-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Elon Musk net worth 2023'}, 'id': 'km0ff2kpc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 122, 'total_tokens': 483}),
 

In [11]:
message = [HumanMessage(content="TravelAgent, what is the waetehr in Hyderabad and Attractions in Hyderabad")]
response = react_graph.invoke({"messages": message})
response

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
##### Final Answer 
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

TravelAgent, what is the waetehr in Hyderabad and Attractions in Hyderabad
================================== Ai Message ==================================
Tool Calls:
  WeatherAgent (g6qn0teys)
 Call ID: g6qn0teys
  Args:
    city: Hyderabad
================================= Tool Message =================================
Name: WeatherAgent

================================== Ai Message ==================================
Tool Calls:
  WeatherAgent (31vr32vgw)
 Call ID: 31vr32vgw
  Args:
    city: Hyderabad
================================= Tool Message =================================
Name: WeatherAgent

================================== Ai Message ==================================
Tool Calls:
  WeatherAgent (46tvvwskb)
 Call ID: 46tvvwskb
  Args:
    city: Hyderabad
================================= Tool Message =================================
Name: WeatherAgent

================================== A